In [1]:
import tools
import cartesian
import racks
import calibration
import samples
import json
import importlib
import random as rnd
import time
import low_level_comm as llc

In [2]:
llc.listSerialPorts()

['COM3', 'COM6', 'COM10', 'COM14']

In [3]:
ar = cartesian.arnie(cartesian_port="COM10", docker_port="COM6")

In [4]:
ar.home()

In [5]:
stp = tools.stationary_touch_probe(ar)

ERROR:root:matchPortsWithDevices(): could not execute finiding device in list.
ERROR:root:matchPortsWithDevices(): Provided patterns_list: ['stationary touch probe']
ERROR:root:None


# Re-calibrating racks

In [21]:
tp = tools.mobile_touch_probe.getTool(ar)

In [14]:
x, y, z = calibration.calibrateStationaryProbe(tp, stp)

In [15]:
p1000_tip_rack = racks.rack(rack_name='p1000_tips_1', rack_type='p1000_tips')
p200_tip_rack = racks.rack(rack_name='p200_tips_1', rack_type='p200_tips')
p20_tip_rack = racks.rack(rack_name='p20_tips_1', rack_type='p20_tips')
ep1 = racks.rack(rack_name='eppendorf_1', rack_type='eppendorf')
magep1 = racks.rack(rack_name='eppendorf_magnetic_1', rack_type='eppendorf_magnetic')
r50 = racks.rack(rack_name='50mL_rack_1', rack_type='50mL')

In [16]:
xr, yr, zr = calibration.calibrateRack(tp, p1000_tip_rack)
ar.move(z=400)
xr, yr, zr = calibration.calibrateRack(tp, p200_tip_rack)
ar.move(z=400)
xr, yr, zr = calibration.calibrateRack(tp, p20_tip_rack)
ar.move(z=400)
xr, yr, zr = calibration.calibrateRack(tp, ep1)
ar.move(z=400)
xr, yr, zr = calibration.calibrateRack(tp, magep1)
ar.move(z=400)
xr, yr, zr = calibration.calibrateRack(tp, r50)
ar.move(z=400)

In [22]:
tp.returnTool()

# Touch wall

In [7]:
ar.openTool()

In [8]:
ar.move(z=0)

In [9]:
p1000 = tools.pipettor.getTool(robot=ar, tool_name='p1000_tool')

In [ ]:
xt, yt, zt = calibration.calibrateTool(p1000, stp)

In [ ]:
p1000.pickUpTip(p1000_tip_rack, 0, 0, raise_z=400)

In [ ]:
ar.move(z=300)

In [29]:
t_ep = samples.sample(sample_name='t_ep_1', sample_type='eppendorf_tube')
t_ep.place(ep1, 0, 3)

In [30]:
p1000.getToSample(t_ep)

In [31]:
p1000.touchWall(t_ep)

In [33]:
t_ep.setVolume(1000)

In [34]:
p1000.touchWall(t_ep)

In [35]:
p1000.touchWall(t_ep, volume=1700)

In [36]:
p1000.dropOffTipToPosition(p1000_tip_rack, 0, 0, raise_z=300, dropoff_dz=-30)

In [37]:
p1000.returnTool()

In [38]:
ar.home()

# Gradual liquid uptake

In [6]:
#p1000 = tools.pipettor.getTool(robot=ar, tool_name='p1000_tool')
p1000 = tools.pipettor(robot=ar, tool_name='p1000_tool')

In [7]:
xt, yt, zt = calibration.calibrateTool(p1000, stp)

In [8]:
p1000_tip_rack = racks.rack(rack_name='p1000_tips_1', rack_type='p1000_tips')

In [9]:
p1000.pickUpTip(p1000_tip_rack, 0, 0, raise_z=400)

In [10]:
ep1 = racks.rack(rack_name='eppendorf_1', rack_type='eppendorf')
t_ep = samples.sample(sample_name='t_ep_1', sample_type='eppendorf_tube')
t_ep.place(ep1, 0, 3)

In [11]:
t_ep.setVolume(1400)

In [16]:
p1000.uptakeLiquid(sample=t_ep, volume=1000)

In [17]:
t_ep.getVolume()

400

In [18]:
p1000.dispenseLiquid(sample=t_ep, volume=1000)

In [19]:
t_ep.getVolume()

1400

In [12]:
p1000.uptakeLiquidGradually(sample=t_ep, volume=1000, dv=100)

In [13]:
t_ep.getVolume()

400

In [14]:
p1000.dropOffTipToPosition(p1000_tip_rack, 0, 0, raise_z=300, dropoff_dz=-40)

In [15]:
ar.home()

# Pipette up and down

In [6]:
p1000 = tools.pipettor(robot=ar, tool_name='p1000_tool')

In [7]:
xt, yt, zt = calibration.calibrateTool(p1000, stp)

In [8]:
p1000_tip_rack = racks.rack(rack_name='p1000_tips_1', rack_type='p1000_tips')

In [9]:
p1000.pickUpTip(p1000_tip_rack, 0, 0, raise_z=400)

In [10]:
ep1 = racks.rack(rack_name='eppendorf_1', rack_type='eppendorf')
t_ep = samples.sample(sample_name='t_ep_1', sample_type='eppendorf_tube')
t_ep.place(ep1, 0, 3)

In [11]:
t_ep.setVolume(1400)

In [12]:
p1000.pipetteUpAndDown(sample=t_ep, uptake_volume=800, repeats=3)

In [13]:
p1000.dropOffTipToPosition(p1000_tip_rack, 0, 0, raise_z=300, dropoff_dz=-40)

In [14]:
ar.home()

# Moving liquid from one tube to the other (with one command)

In [6]:
p1000 = tools.pipettor(robot=ar, tool_name='p1000_tool')

In [7]:
xt, yt, zt = calibration.calibrateTool(p1000, stp)

In [8]:
p1000_tip_rack = racks.rack(rack_name='p1000_tips_1', rack_type='p1000_tips')

In [9]:
p1000.pickUpTip(p1000_tip_rack, 0, 0, raise_z=400)

In [43]:
ep1 = racks.rack(rack_name='eppendorf_1', rack_type='eppendorf')
t_ep = samples.sample(sample_name='t_ep_1', sample_type='eppendorf_tube')
t_ep.place(ep1, 0, 3)

In [44]:
t50 = samples.sample(sample_name='t50_1', sample_type='50mL_tube')
r50 = racks.rack(rack_name='50mL_rack_1', rack_type='50mL')
t50.place(r50, 2, 0)

In [45]:
t_ep.setVolume(0)
t50.setVolume(50000)

In [13]:
p1000.moveLiquid(t50, t_ep, 700)

In [14]:
p1000.moveLiquid(t_ep, t50, 700)

In [46]:
sample_list = [samples.sample(sample_name='t_ep_'+str(x), sample_type='eppendorf_tube') for x in range(5)]

In [47]:
i = 0
for sample in sample_list:
    sample.place(ep1, i, 3)
    i += 1

In [48]:
[x.sample_data['x_well'] for x in sample_list]

[0, 1, 2, 3, 4]

In [28]:
volume_list = [100, 347, 901, 178, 333]

In [19]:
p1000.distributeLiquid(t50, sample_list, volume_list)

Remains to pipette: 1859 Current volume:  100 Well:  0
Remains to pipette: 1759 Current volume:  347 Well:  1
Remains to pipette: 1412 Current volume:  901 Well:  2
Remains to pipette: 511 Current volume:  178 Well:  3
Remains to pipette: 333 Current volume:  333 Well:  4


In [49]:
def distributeLiquid(tool, sample_origin, sample_destination_list, vol_list,
                         uptake_delay=0, release_delay=0, immerse_vol_origin=None,
                         immerse_volume_destination=None, touch_wall=False):
    vol_in_tip = 0
    remaining_vol_to_move = sum(vol_list)
    vol_to_move_plunger = 0
    for sample_destination, volume in zip(sample_destination_list, vol_list):
        print ("Remains to pipette:", remaining_vol_to_move, "Current volume: ", volume, "Well: ", sample_destination.sample_data['x_well'])
        # Since tool is moving plunger to absolute position, every next sample will have added "volume"
        # to pipette
        vol_to_move_plunger = vol_to_move_plunger + volume
        if vol_in_tip < volume:
            tool.getToSample(sample=sample_origin)
            tool.movePlunger(-40)
            vol_in_tip = 0
            tool.touchWall(sample=sample_origin)
            # Lifting up again
            tool.getToSample(sample=sample_origin)
            if remaining_vol_to_move > tool.max_allowed_vol:
                vol_to_uptake = tool.max_allowed_vol
            else:
                vol_to_uptake = remaining_vol_to_move
            print ("Uptaking volume: ", vol_to_uptake)
            tool.uptakeLiquid(sample=sample_origin, volume=vol_to_uptake,
                                  uptake_delay=uptake_delay)
            vol_in_tip = vol_to_uptake
            # Resetting plunger movement after fresh liquid uptake
            vol_to_move_plunger = volume
        print ("Dispensing volume: ", volume)
        tool.dispenseLiquid(sample=sample_destination, volume=vol_to_move_plunger, release_delay=release_delay,
                                plunger_retract=False)
        vol_in_tip = vol_in_tip - volume
        if touch_wall:
                tool.touchWall(sample=sample_destination)
        remaining_vol_to_move = remaining_vol_to_move - volume

In [29]:
distributeLiquid(p1000, t50, sample_list, volume_list)

Remains to pipette: 1859 Current volume:  100 Well:  0
Uptaking volume:  1000.0
Remains to pipette: 1759 Current volume:  347 Well:  1
Remains to pipette: 1412 Current volume:  901 Well:  2
Uptaking volume:  1000.0
Remains to pipette: 511 Current volume:  178 Well:  3
Uptaking volume:  511
Remains to pipette: 333 Current volume:  333 Well:  4


In [50]:
volume_list = [100, 200, 300, 200, 50]
sum(volume_list)

850

In [51]:
distributeLiquid(p1000, t50, sample_list, volume_list)

Remains to pipette: 850 Current volume:  100 Well:  0
Uptaking volume:  850
Dispensing volume:  100
Remains to pipette: 750 Current volume:  200 Well:  1
Dispensing volume:  200
Remains to pipette: 550 Current volume:  300 Well:  2
Dispensing volume:  300
Remains to pipette: 250 Current volume:  200 Well:  3
Dispensing volume:  200
Remains to pipette: 50 Current volume:  50 Well:  4
Dispensing volume:  50


In [30]:
178+333

511

In [31]:
ar.move(z=300)

In [52]:
for sample in sample_list:
    p1000.uptakeLiquid(sample, 1000, uptake_delay=0, immerse_volume=30)
    p1000.dispenseLiquid(t50, 1000, immerse_volume=60000)
    p1000.uptakeLiquid(sample, 80, uptake_delay=3, immerse_volume=1)
    p1000.dispenseLiquid(t50, 100, immerse_volume=60000)

In [32]:
p1000.movePlunger(-40)

In [33]:
p1000.movePlunger(0)

In [18]:
p1000.dropOffTipToPosition(p1000_tip_rack, 0, 0, raise_z=300, dropoff_dz=-40)

In [55]:
p1000.returnTool()

In [19]:
ar.home()

# Testing procedure to uptake all liquid from bottom

In [6]:
p1000 = tools.pipettor(robot=ar, tool_name='p1000_tool')

In [41]:
xt, yt, zt = calibration.calibrateTool(p1000, stp)

In [8]:
p1000_tip_rack = racks.rack(rack_name='p1000_tips_1', rack_type='p1000_tips')

In [42]:
p1000.pickUpTip(p1000_tip_rack, 0, 0, raise_z=400)

In [10]:
ep1 = racks.rack(rack_name='eppendorf_1', rack_type='eppendorf')
t_ep = samples.sample(sample_name='t_ep_1', sample_type='eppendorf_tube')
t_ep.place(ep1, 0, 3)

In [11]:
t50 = samples.sample(sample_name='t50_1', sample_type='50mL_tube')
r50 = racks.rack(rack_name='50mL_rack_1', rack_type='50mL')
t50.place(r50, 2, 0)

In [12]:
t_ep.setVolume(0)
t50.setVolume(50000)

In [13]:
sample_list = [samples.sample(sample_name='t_ep_'+str(x), sample_type='eppendorf_tube') for x in range(5)]

In [14]:
i = 0
for sample in sample_list:
    sample.place(ep1, i, 3)
    i += 1

In [15]:
[x.sample_data['x_well'] for x in sample_list]

[0, 1, 2, 3, 4]

In [16]:
volume_list = [100, 347, 901, 178, 333]

In [20]:
p1000.distributeLiquid(t50, sample_list, volume_list, touch_wall=True)

In [21]:
for sample, volume in zip(sample_list, volume_list):
    p1000.uptakeLiquid(sample, 1000, uptake_delay=0.3)
    p1000.dispenseLiquid(t50, volume, immerse_volume=60000, blow_extra=True)

## Finding pipetting parameters to empty the tube

In [19]:
p1000.moveLiquid(t50, t_ep, 900)

In [20]:
p1000.uptakeLiquid(t_ep, 1000, uptake_delay=2, retract_z_speed=20, bottom_gap=5)

In [21]:
p1000.dispenseLiquid(t50, 1000, immerse_volume=60000, blow_extra=True)

In [25]:
def sideTubeUptake(tool, extra_vol, uptake_delay, uptake_vol, axis, distance):
    extra_vol = extra_vol - uptake_vol
    tool.robot.moveAxisDelta(axis=axis, value=distance)
    tool.movePlungerToVol(extra_vol)
    time.sleep(uptake_delay)
    tool.robot.moveAxisDelta(axis=axis, value=-distance)
    return extra_vol

In [30]:
def uptakeAll(tool, sample, tip_ignore=False, volume=None, extra_vol_frac=0.2, 
              immerse_levels_list=[10, 0], side_uptake_radius=2,  uptake_delay=0):
    """
    This function tries to uptake all the liquid that the tube may have, drying it up completely.
    """
    if (not tool.tip_attached) and (not tip_ignore):
        print("ERROR: pipette tip was not attached.")
        return
        
    # Moving robot towards the sample
    tool.getToSample(sample=sample)
    
    # Figuring out volume
    if volume is None:
        volume = sample.getVolume()
    tool_max_vol = tool.max_allowed_vol
    extra_vol = tool_max_vol * extra_vol_frac
    # Moving plunger down
    tool.movePlungerToVol(volume+extra_vol)
    # Immersing into the tube
    z = sample.sampleVolToZ(volume=immerse_levels_list[0], tool=tool)
    tool.robot.move(z=z)
    # Uptaking most of the liquid volume (until extra volume mark)
    tool.movePlungerToVol(extra_vol)
    time.sleep(uptake_delay)
    # Now going through the rest of the immersing levels
    vol_uptake_per_cycle = extra_vol / (len(immerse_levels_list[1:]) * 1.0)
    vol_uptake_per_suck = vol_uptake_per_cycle / 5.0
    for level in immerse_levels_list[1:]:
        z = sample.sampleVolToZ(volume=level, tool=tool)
        tool.robot.move(z=z)
        # Sucking at the center
        extra_vol = sideTubeUptake(tool, extra_vol, uptake_delay/5.0, vol_uptake_per_suck, 'x', 0)
        # Sucking at the north
        extra_vol = sideTubeUptake(tool, extra_vol, uptake_delay/5.0, vol_uptake_per_suck, 'y', -side_uptake_radius)
        # Sucking at the south
        extra_vol = sideTubeUptake(tool, extra_vol, uptake_delay/5.0, vol_uptake_per_suck, 'y', side_uptake_radius)
        # Sucking at the west
        extra_vol = sideTubeUptake(tool, extra_vol, uptake_delay/5.0, vol_uptake_per_suck, 'x', -side_uptake_radius)
        # Sucking at the east
        extra_vol = sideTubeUptake(tool, extra_vol, uptake_delay/5.0, vol_uptake_per_suck, 'x', side_uptake_radius)
    sample.setVolume(0)
    

In [38]:
p1000.dropOffTipToPosition(p1000_tip_rack, 0, 0, raise_z=300, dropoff_dz=-40)

In [40]:
ar.home()

In [27]:
p1000.pickUpTip(p1000_tip_rack, 0, 0, raise_z=400)

In [43]:
p1000.moveLiquid(t50, t_ep, 500)

In [44]:
uptakeAll(p1000, t_ep)

In [36]:
uptakeAll(p1000, t_ep, volume=1000)

In [45]:
p1000.dispenseLiquid(t50, 1000, immerse_volume=60000, blow_extra=True)